# --- Day 19: An Elephant Named Joseph ---

The Elves contact you over a highly secure emergency channel. Back at the North Pole, the Elves are busy misunderstanding White Elephant parties.

Each Elf brings a present. They all sit in a circle, numbered starting with position 1. Then, starting with the first Elf, they take turns stealing all the presents from the Elf to their left. An Elf with no presents is removed from the circle and does not take turns.

For example, with five Elves (numbered 1 to 5):

```
  1
5   2
 4 3
```

- Elf 1 takes Elf 2's present.
- Elf 2 has no presents and is skipped.
- Elf 3 takes Elf 4's present.
- Elf 4 has no presents and is also skipped.
- Elf 5 takes Elf 1's two presents.
- Neither Elf 1 nor Elf 2 have any presents, so both are skipped.
- Elf 3 takes Elf 5's three presents.

So, with five Elves, the Elf that sits starting in position 3 gets all the presents.

With the number of Elves given in your puzzle input, which Elf gets all the presents?

---

this... looks hard. In fact it looks like something solvable quite easily using math. But here goes a non mathy approach.

In [1]:
puzzle_input = 3014603

First up, we need a circular list, so importing likely candidates to make a circular list:

In [8]:
from collections import deque, namedtuple
from itertools import cycle
import numpy as np

Elf = namedtuple("Elf", ["Position", "Presents"])
[Elf(i, 1) for i in range(3)]

[Elf(Position=0, Presents=1),
 Elf(Position=1, Presents=1),
 Elf(Position=2, Presents=1)]

I like named tuples, but for part one we only need to know the last elf so I'm just using lists.

The below simulates the gift passing until there is only one elf left:

In [16]:
def last_elf(n=5, verbose=False):
    """returns the elf with all the presents"""
    elves = deque([i for i in range(1,n+1)])
    if verbose: print(elves)
        
    while len(elves) > 1:
        
        # only circle around if the list is odd
        pop = False if len(elves) % 2 == 0 else True
        
        # removing all the even elves as their gifts have been grabbed by the odd 
        # elf to their left
        elves = deque([e for i,e in enumerate(elves) if i%2==0])
        if verbose: print(elves)
        
        # for odd length lists, the last elf grabs the presents from the first elf
        if len(elves) > 1 and pop:
            elves.popleft()
            if verbose: print(elves)
            
    return elves[0]

last_elf(n=5, verbose=True)

deque([1, 2, 3, 4, 5])
deque([1, 3, 5])
deque([3, 5])
deque([3])


3

In [18]:
%time last_elf(puzzle_input)

CPU times: user 1.08 s, sys: 4 ms, total: 1.08 s
Wall time: 1.1 s


1834903

`1834903` is the answer for part 1.

# --- Part Two ---

Realizing the folly of their present-exchange rules, the Elves agree to instead steal presents from the Elf directly across the circle. If two Elves are across the circle, the one on the left (from the perspective of the stealer) is stolen from. The other rules remain unchanged: Elves with no presents are removed from the circle entirely, and the other elves move in slightly to keep the circle evenly spaced.

So I thought using a numpy array would speed things up but turns out it doesnt:

In [156]:
def last_elf_2(n=5, verbose=False):
    """returns the elf with all the presents"""
    
    elves = np.arange(1, n+1)
    i = 0
    num_elves = len(elves)
    
    while num_elves > 1:
        if verbose: print(i, elves)
        
        if num_elves == 2:
            return elves[0]
        
        elves = np.delete(elves, num_elves//2)
        elves = np.roll(elves,-1)
        num_elves = len(elves)
        
        i += 1
        if i % 1000 == 0:
            print(f"Loop {i}, elves left: {num_elves}")
         
    return "Shouldn't have reached here"

%time last_elf_2(n=5, verbose=True) # answer for n=5 is 2

0 [1 2 3 4 5]
1 [2 4 5 1]
2 [4 1 2]
3 [2 4]
CPU times: user 3 ms, sys: 0 ns, total: 3 ms
Wall time: 2.38 ms


2

the above solution is way too slow for bigger inputs, probably since I'm deleting from the middle of the array. So as faster way to do this is to use two queues, implemented using deque since that does fast pops from left and right.

In [155]:
def part_2_faster(n=5, verbose=False):
    """returns the elf with all the presents"""
    
    left = deque([i for i in range(1, (n//2)+1)])
    right = deque([i for i in range((n//2)+1, n+1)])
    
    num_elves = len(left) + len(right)
    
    while num_elves > 1:
        
        if num_elves == 2:
            return left[0]
        
        if verbose: print("start:", left, right)
        
        if len(left) > len(right):
            left.pop()
        else:
            right.popleft()
        
        if verbose: print("remove:", left, right)
        
        # rotate 1 letter backwards
        right.append(left.popleft())
        left.append(right.popleft())
        
        if verbose: print("rotate:", left, right)
        
        num_elves = len(left) + len(right)
    
    return "something bad happened"

%time part_2_faster(n=5, verbose=True) # answer for n=5 is 2

start: deque([1, 2]) deque([3, 4, 5])
remove: deque([1, 2]) deque([4, 5])
rotate: deque([2, 4]) deque([5, 1])
start: deque([2, 4]) deque([5, 1])
remove: deque([2, 4]) deque([1])
rotate: deque([4, 1]) deque([2])
start: deque([4, 1]) deque([2])
remove: deque([4]) deque([2])
rotate: deque([2]) deque([4])
CPU times: user 2 ms, sys: 1 ms, total: 3 ms
Wall time: 2.24 ms


2

In [151]:
%time part_2_faster(puzzle_input)

CPU times: user 3.31 s, sys: 29 ms, total: 3.34 s
Wall time: 3.36 s


1420280

`1420280` is the right answer

## Notes:

- deque is fast
- see pattern of data and whether multiple lists make sense, or making my own data structure like a linked list or tree
- np.roll() is great but not if my list to be rolled is split into two. So made my own version of np.roll